In [34]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
# Load train and test data
df_train = pd.read_csv("avila-tr.txt", header=None , names=['F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','Class']);
df_test = pd.read_csv("avila-ts.txt", header=None , names=['F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','Class']);

In [4]:
df_test

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,Class
0,-3.498799,0.250492,0.232070,1.224178,-4.922215,1.145386,0.182426,-0.165983,-0.123005,1.087144,W
1,0.204355,-0.354049,0.320980,0.410166,-0.989576,-2.218127,0.220177,0.181844,2.090879,-2.009758,A
2,0.759828,-1.304042,-0.023991,-0.973663,-0.006417,-0.349509,-0.421580,-0.450127,0.469443,0.060952,I
3,-0.005490,0.360409,0.281860,-0.213479,-1.168333,-1.013906,-0.346080,1.176165,0.968347,-0.627999,E
4,0.080916,0.101320,0.104040,0.140490,0.261718,0.480988,0.710932,-0.253430,-0.497183,0.155681,A
...,...,...,...,...,...,...,...,...,...,...,...
10432,-0.128929,-0.040001,0.057807,0.557894,0.261718,-0.930856,-0.044076,1.158458,2.277968,-0.699884,X
10433,0.266074,0.556689,-0.020434,0.176624,0.261718,-0.515608,0.597681,0.178349,0.625350,-0.657245,G
10434,-0.054866,0.580242,0.032912,-0.016668,0.261718,1.519109,0.371178,-0.985508,-0.403638,1.276301,A
10435,0.080916,0.588093,0.015130,0.002250,0.261718,-0.930856,-0.270579,0.163807,-0.091823,-0.593329,F


In [5]:
# As data has been already normalized we do not look into standardizing our data
# As our data is already in numerical form we also don't look into cleaning our data.

In [6]:
# Since classes are imbalanced we could use class weights during learning
# to provide some bias to minority classes without having to resample our data

In [7]:
X_train = df_train.drop(axis=1,labels="Class");
y_train = df_train['Class'];

X_test = df_test.drop(axis=1,labels="Class");
y_test = df_test['Class'];

In [8]:
y_train_labels = pd.get_dummies(y_train)

In [9]:
y_train_labels['A']

0        1
1        1
2        1
3        1
4        0
        ..
10425    0
10426    0
10427    1
10428    0
10429    0
Name: A, Length: 10430, dtype: uint8

In [17]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras import optimizers
from keras.utils.vis_utils import plot_model
np.random.seed(1);
from keras.callbacks import EarlyStopping

In [11]:
y = y_train_labels.to_numpy();
np.unique(y_train)

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'W', 'X', 'Y'],
      dtype=object)

In [12]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

C:\Users\Maxime\Anaconda3\lib\site-packages\sklearn\utils\validation.py:71: FutureWarning: Pass classes=['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'W' 'X' 'Y'], y=0        A
1        A
2        A
3        A
4        F
        ..
10425    F
10426    F
10427    A
10428    E
10429    X
Name: Class, Length: 10430, dtype: object as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [13]:
class_weight

array([  0.20279204, 173.83333333,   8.43851133,   2.46922348,
         0.79375951,   0.44322625,   1.94880419,   1.67469493,
         1.0459286 ,  19.75378788,   1.66507024,   3.26754386])

In [14]:
y = y.astype('float32');

In [15]:
# Learning rate of 0.1 after multiple tries between 0.01 and 2
# 3 couches cachés
# 200

In [32]:
n_h=200; n_o=12; lrate=0.1; nepochs=50; isVerbose=1;

n_i = X_train.shape[1] # 10 distinct features in our data
model = Sequential()
model.add(Dense(n_h, input_dim=n_i, activation="relu"));
model.add(Dense(n_h, input_dim=n_h, activation="relu"));
model.add(Dense(n_h, input_dim=n_h, activation="relu"));
model.add(Dense(n_o, activation = "softmax"));

model.compile(optimizer=optimizers.SGD(lr=lrate), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(
    X_train, 
    y, 
    epochs=nepochs, 
    batch_size=32,
    verbose=isVerbose, 
    class_weight=class_weight,
    #validation_split=0.4,
    #callbacks=[EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)],
)

Epoch 1/50
10430/10430 [==============================] - 1s 60us/step - loss: 1.3697 - accuracy: 0.5453
Epoch 2/50
10430/10430 [==============================] - 1s 51us/step - loss: 0.9982 - accuracy: 0.6528
Epoch 3/50
10430/10430 [==============================] - 1s 52us/step - loss: 0.9176 - accuracy: 0.6738
Epoch 4/50
10430/10430 [==============================] - 1s 51us/step - loss: 0.7925 - accuracy: 0.7032
Epoch 5/50
10430/10430 [==============================] - 1s 55us/step - loss: 0.7677 - accuracy: 0.7176
Epoch 6/50
10430/10430 [==============================] - 0s 47us/step - loss: 0.6887 - accuracy: 0.7376
Epoch 7/50
10430/10430 [==============================] - 0s 48us/step - loss: 0.6422 - accuracy: 0.7490
Epoch 8/50
10430/10430 [==============================] - 1s 59us/step - loss: 0.5958 - accuracy: 0.7690
Epoch 9/50
10430/10430 [==============================] - 1s 48us/step - loss: 0.6251 - accuracy: 0.7678
Epoch 10/50
10430/10430 [==============================

In [29]:
predicted = model.predict_classes(X_test)

In [30]:
# Convert y_test to values
y_test_valued = []
for entry in y_test:
    if (entry == 'A'):
        y_test_valued.append(0);
    elif (entry == 'B'):
        y_test_valued.append(1);
    elif (entry == 'C'):
        y_test_valued.append(2);
    elif (entry == 'D'):
        y_test_valued.append(3);
    elif (entry == 'E'):
        y_test_valued.append(4);
    elif (entry == 'F'):
        y_test_valued.append(5);
    elif (entry == 'G'):
        y_test_valued.append(6);
    elif (entry == 'H'):
        y_test_valued.append(7);
    elif (entry == 'I'):
        y_test_valued.append(8);
    elif (entry == 'W'):
        y_test_valued.append(9);
    elif (entry == 'X'):
        y_test_valued.append(10);
    elif (entry == 'Y'):
        y_test_valued.append(11);
        

In [36]:
count = 0;
N = len(y_test_valued);
for index, val in enumerate(y_test_valued):
    if (predicted[index] == y_test_valued[index]):
        count+=1;
        
print("Testset Accuracy: ", count / N)

Testset Accuracy:  0.9251700680272109


In [19]:
# Even though we chose some very high large values for number of hidden layers
# number of neurones, and a somewhat large number of epochs
# We were still able to get a precision of 93+%
# Which is why we could say that we did not overfit our sample.